In [1]:
from vnstock import *
from datetime import date, datetime, timedelta
from vnstock.chart import *
import plotly.io as pio
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as mticker
from mplfinance.original_flavor import candlestick_ochl
import math
from matplotlib.dates import date2num
from matplotlib.dates import DateFormatter, MonthLocator
from mplfinance.original_flavor import candlestick_ohlc

In [ ]:
stock_codes = ['VCG', 'HSG', 'SSI', 'MWG', 'GIL', 'DXG']

In [36]:
df = pd.read_csv('./Data/GIL.csv', index_col='time')
df = df[['ticker', 'open', 'high', 'low', 'close' ,'latest_change', 'volume']]
df['date'] = df.index

df = df[~df.index.duplicated(keep='first')]


In [43]:
def read_data_ohlc(filepath):

    ## READ FILE
    df = pd.read_csv(filepath, index_col='time')
    df = df[~df.index.duplicated(keep='first')]

    df = df[['ticker', 'open', 'high', 'low', 'close' ,'latest_change', 'volume']]
    df['date'] = df.index
    df.index = pd.DatetimeIndex(df.index)
    df['MA20'] = df['close'].rolling(20).mean()
    df['MA50'] = df['close'].rolling(50).mean()
    df['MA200'] = df['close'].rolling(200).mean()
    try: 
        df['RSI'] = compute_RSI(df['close'], 14)
        print("GET RSI SUNCCESSFUL!")
    except:
        print("ERROR! GET RSI UNSUCCESSFUL")

    latest_info = df.iloc[-1, :]
    latest_price = latest_info['close']
    latest_change = str(latest_info['latest_change'])\
        if (latest_info['latest_change'][0] == '-') else '+' + str(latest_info['latest_change'])
    latest_volume = latest_info['volume']

    return df, latest_price, latest_change, latest_volume

In [44]:
data_ohlc = read_data_ohlc('./Data/GIL.csv')
data_ohlc

GET RSI SUNCCESSFUL!


(           ticker   open   high    low  close     latest_change   volume  \
 time                                                                       
 2022-01-05    GIL  56370  59620  55950  59280     3250.0 (5.8%)   753100   
 2022-01-06    GIL  59700  60640  58940  59540     260.0 (0.44%)   693600   
 2022-01-07    GIL  59280  59540  58090  58940   -600.0 (-1.01%)   209900   
 2022-01-10    GIL  58090  60730  56970  58940        0.0 (0.0%)   627600   
 2022-01-11    GIL  59190  59790  57230  58250   -690.0 (-1.17%)   272300   
 ...           ...    ...    ...    ...    ...               ...      ...   
 2023-10-24    GIL  27300  27850  27100  27850     550.0 (2.01%)   358400   
 2023-10-25    GIL  28200  28350  27600  27700   -150.0 (-0.54%)   382600   
 2023-10-26    GIL  27200  27200  25800  25800  -1900.0 (-6.86%)   859000   
 2023-10-27    GIL  25300  25900  24000  24700  -1100.0 (-4.26%)  1286600   
 2023-10-30    GIL  24700  25000  23900  23900   -800.0 (-3.24%)   266800   

In [4]:
def combine_columns(row):
    return f"{row['change']} ({row['%_change']}%)"

In [5]:
# Lấy dữ liệu cổ phiếu

def real_time_stock(stock_code):
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    stock_hist = stock_historical_data(stock_code, "2022-01-01",
                                         str(date.today()), "1D", "stock")
    stock_hist["%_change"] = round(((stock_hist['close'] - stock_hist['close'].shift(1))
                                    / stock_hist['close'].shift(1)) * 100, 2)
    stock_hist['change'] = (stock_hist['close'] - stock_hist['close'].shift(1))
    stock_hist['latest_change'] = stock_hist.apply(combine_columns, axis=1)
    data = stock_hist.iloc[-1]
    # return (str(time_stamp), data['close'], data['volume'], data['%_change'], data['change'])
    return stock_hist

In [ ]:
for stock in stock_codes:
  stock_data = real_time_stock(stock)
  index_with_nan = stock_data.index[stock_data.isnull().any(axis=1)]
  stock_data.drop(index_with_nan, inplace=True)
  stock_data.to_csv(f'{stock}.csv', index=False, header=True)
  print(stock_data)

In [ ]:
# Vẽ biểu đồ

def real_time_chart(stock_code):
    stock = stock_historical_data(stock_code, "2022-01-01", str(date.today()), "1D", "stock")
    fig = candlestick_chart(stock, ma_periods=[20, 50, 200], show_volume=True, reference_period=300, figure_size=(15, 8), 
                        title=f'{stock_code} - Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', 
                        colors=('lightgray', 'gray'), reference_colors=('black', 'blue'))
    # pio.write_image(fig, 'plot.png')
    fig.show()

In [ ]:
# real_time_chart('VHC')
stock = stock_historical_data('VHC', "2022-01-01", str(date.today()), "1D", "stock")
stock

In [ ]:
fig = candlestick_chart(stock, ma_periods=[20, 50, 200], show_volume=True, reference_period=300, figure_size=(15, 8), 
                    title=f'VHC - Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', 
                    colors=('lightgray', 'gray'), reference_colors=('black', 'blue'))
fig.show()